In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from typing import Tuple

from roveranalyzer.utils import Project
import contextily as ctx
from shapely.geometry import box, Point
from roveranalyzer.analysis import OppAnalysis, DensityMap
import roveranalyzer.simulators.crownet.dcd as Dmap
import roveranalyzer.simulators.opp as OMNeT
import folium
from folium.plugins import TimestampedGeoJson, TimeSliderChoropleth, HeatMapWithTime

def run_data(run, sim, hdf_file="data.h5") -> Tuple[Dmap.DcdHdfBuilder, OMNeT.CrownetSql]:
    data_root = f"{os.environ['HOME']}/repos/crownet/crownet/simulations/{sim}/results/{run}"
    builder = Dmap.DcdHdfBuilder.get(hdf_file, data_root).epsg(Project.UTM_32N)
    
    sql = OMNeT.CrownetSql(
            vec_path=f"{data_root}/vars_rep_0.vec", 
            sca_path=f"{data_root}/vars_rep_0.sca", 
        network="World")
    return data_root, builder, sql


# Example Müncher Freiheit with ~90 Pedestrians using Vadere

In [ ]:
# position offset?
root_path1, b1, sql1 = run_data("vadere_XX_20211129-11:44:47", "mucFreiheitLte")
p1 = b1.build(override_hdf=True)

In [ ]:
m = p1.map_p
i = pd.IndexSlice
o = m.operators
m.add_filter(selection=o.GT(1))

In [ ]:
nodes1 = sql1.host_position(epsg_code_base=Project.UTM_32N, epsg_code_to=Project.OpenStreetMaps)
cells1 = p1.global_p.geo(Project.OpenStreetMaps).get_dataframe()

In [ ]:
def get_map(cells, nodes, time=None):
    m = DensityMap.get_interactive(
        cells = cells,
        nodes = nodes, 
        time = time)
    return m

def get_node_map(map_p, nodes, time, node_id):
    i = pd.IndexSlice
    cells = map_p.geo(Project.OpenStreetMaps)[i[time, :, :, :, node_id]]
    cells = cells[cells["selection"] == 1]
    _nodes = nodes[nodes["time"] == time]
    return get_map(cells, _nodes)


In [ ]:
m_10 = get_map(cells1, nodes1, 10.0)
m_10.save("m10.html")

m_60 = get_map(cells1, nodes1, 60.0)
m_60.save("m60.html")

m_90 = get_map(cells1, nodes1, 90.0)
m_90.save("m90.html")

In [ ]:
nid = 2356
# nid = 2236
m_node_10 = get_node_map(p1.map_p, nodes1, 10.0, nid)
m_node_10.save("m_node_10.html")

m_node_60 = get_node_map(p1.map_p, nodes1, 60.0, nid)
m_node_60.save("m_node_60.html")

m_node_90 = get_node_map(p1.map_p, nodes1, 90.0, nid)
m_node_90.save("m_node_90.html")

In [ ]:
dcd = b1.build_dcdMap()

22.3 s

In [ ]:
dcd.plot_(savefig=f"{root_path1}/fig/count_diff3.pdf")

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(16,9))

ntable, ntable_index = OppAnalysis.get_neighborhood_table_size(sql=sql1)
OppAnalysis.plot_neighborhood_table_size_over_time(axes[0], ntable, ntable_index)

pkt_source = OppAnalysis.get_packet_source_distribution(sql=sql1, app_path=".app[1].app")
OppAnalysis.plot_packet_source_distribution(axes[1], data=pkt_source)
axes[1].get_legend().remove()

fig.savefig(f"{root_path1}/fig/figure_2.pdf")


# Stationary Example

In [ ]:
builder_stat, sql_stat = run_data("mucStationaryTest_lowerEnb_20211119-13:51:16", "mucFreiheitLte")
p2 = builder_stat.build()
nodes_stat = sql_stat.host_position(epsg_code_base=Project.UTM_32N, epsg_code_to=Project.OpenStreetMaps)
cells_stat = p2.global_p.geo(Project.OpenStreetMaps).get_dataframe()
map_global_stat = get_map(cells_stat, nodes_stat)
map_global_stat.save("map_global_stationary.html")
map_global_stat

# Interactive Maps

In [ ]:

b1, sql1 = run_data("mucStationaryTest_lowerEnb_20211119-13:51:16", "mucFreiheitLte")
b2, sql2 = run_data("mucSumo_base_20211123-17:58:12", "mucFreiheitLte")

# p2= b2.build(override_hdf=False)
# pos = sql2.host_position(epsg_code=Project.UTM_32N)
# pos = pos.to_crs(epsg=Project.OpenStreetMaps.replace("EPSG:", ""))

ped_r = 0.3
pos = sql2.host_position(epsg_code=Project.UTM_32N)
g = [ box(x-ped_r, y-ped_r, x+2*ped_r, y+2*ped_r) for x, y in zip(pos["x"], pos["y"])]
pos["geometry"] = g
# pos = pos.to_crs(epsg=Project.OpenStreetMaps.replace("EPSG:", ""))

# pos = pos.drop_duplicates()
# pos = pos.reset_index(drop=True)
# pos.index.name = "obj_id"
# sql2.host_ids()

Add feature_id, color and opacity values to all positions.

In [ ]:
pos.index.name = "feature_id"
pos["color"] = "#6106ff"
pos["opacity"] = 1.0
pos = pos.reset_index().set_index(["feature_id", "time"])

Create product of all features and time steps to create entries in style_dict that will set the opacity of the feature to `0.0`
when the feature should not be shown (wrong time)

In [ ]:
import itertools
feature_time = itertools.product()
feature_time_idx = pd.MultiIndex.from_product(
    [pos.index.get_level_values(0), pos.index.get_level_values(1)], 
    names=["feature_id", "time"]
)
df_missing = pd.DataFrame(index=feature_time_idx.difference(pos.index), columns=["color", "opacity"])
df_missing["color"] = "#6106ff"
df_missing["opacity"] = "0.0"


Create style_dict of the form 
```
{
    'feature-id-0' : {
        '<time-stamp-1> : {color: xxx, opacity: yyy},
        '<time-stamp-2> : {color: xxx, opacity: yyy},
        '<time-stamp-3> : {color: xxx, opacity: yyy},
    },
    ...
    'feature-id-(N-1)' : {
        '<time-stamp-1> : {color: xxx, opacity: yyy},
        '<time-stamp-2> : {color: xxx, opacity: yyy},
        '<time-stamp-3> : {color: xxx, opacity: yyy},
    },
}
```

In [ ]:
style_df  = pd.concat([pos.loc[:, ["color", "opacity"]], df_missing])
style_df.sort_index()
style_dict = {}
for feature_id in style_df.index.get_level_values("feature_id"):
    t_dict = {}
    for time, data in style_df.loc[feature_id].iterrows():
        t_dict[str(time)] = data.to_dict()
    style_dict[str(feature_id)] = t_dict



In [ ]:
pos.head()

In [ ]:
map = folium.Map([48.1596113,11.58011], tiles="Stamen Toner", zoom_start=18, max_zoom=22)
# map = folium.Map([0, 0], tiles="Stamen Toner", zoom_start=2)

g = TimeSliderChoropleth(
    data = pos.reset_index(),
    styledict= style_dict, 
    overlay= True
)

# g._template = template
g.add_to(map)

folium.GeoJson(
    data=pos.reset_index(), 
    style_function= lambda x : {'opacity': 0.0, 'fillColor': 'red', 'fillOpacity': 0.0, 'color': None},
    tooltip=folium.GeoJsonTooltip(['host', 'time']), 
    popup= folium.GeoJsonPopup(['time', 'hostId', 'host'])
).add_to(map)

DensityMap.folium.add_google_tile(map).add_control(map)
map.save("postion_map.html")
map

In [ ]:
pos.reset_index()

In [ ]:
pos3 = sql2.host_position(epsg_code=Project.UTM_32N)
pos3 = pos3.to_crs(epsg=Project.OpenStreetMaps.replace("EPSG:", ""))
pos3 = pos3.to_crs(crs=Project.WSG84_lat_lon)
pos3["lat"] = pos3["geometry"].y
pos3["lon"] = pos3["geometry"].x
# pos3["time"] = np.floor(pos3["time"])
pos3.head()


In [ ]:
time = pos3["time"].unique()
time.sort()
lat_lon = []
for t in time:
    tmp = []
    for i, data in pos3[pos3["time"] == t].iterrows():
        tmp.append([data["lat"], data["lon"]])
    lat_lon.append(tmp)
    # print(f"{t}: {len(tmp)}")

In [ ]:

m = DensityMap.get_interactive(nodes=pos3[pos3["time"]==8.0])
m.save("peds_t8.html")

In [ ]:
map = folium.Map([48.1596113,11.58011], tiles="Stamen Toner", zoom_start=18, max_zoom=22)
HeatMapWithTime(
    lat_lon, 
    radius=10, 
    auto_play= True, 
    position="bottomright", 
    min_opacity = 0.9, 
    max_opacity = 1.0,
    # scale_radius= True,
    gradient={0.1: '#ffb3ac', .4: '#ff755d', 1: '#ff0202'}
).add_to(map)


folium.LayerControl().add_to(map)
map.save("density_map.html")
map

In [ ]:
import datetime
from typing import Dict
style_dict: Dict = {}
t_df = pos.reset_index().set_index("time")
for i in range(t_df.shape[0]):
    obj_id = str(t_df.iloc[i]["obj_id"])
    time_dict = style_dict.get(obj_id, {})
    time = str(1637693844.451419 + t_df.index[i])
    time_dict[time] = dict(color="red", opacity=1.0)
    style_dict[obj_id] = time_dict

style_dict['0']

time = str(1637693844.451419 + t_df.index[i])
# datetime.datetime.fromtimestamp()

In [ ]:
map = folium.Map([48.1596113,11.58011], tiles="Stamen Toner", zoom_start=18, max_zoom=22)
# map = folium.Map([0, 0], tiles="Stamen Toner", zoom_start=2)

g = TimeSliderChoropleth(
    data = pos,
    styledict= style_dict, 
    overlay= True
).add_to(map)

# pos2.explore(
#     m=map,
#     name = "fpp",
#     color= "red"
# )

folium.LayerControl().add_to(map)
# map.save("out.hml")
map


In [ ]:
from shapely.geometry import box
pos2 = sql2.host_position(epsg_code=Project.UTM_32N)
g = [ box(x, y, x+3, y+3) for x, y in zip(pos2["x"], pos2["y"])]
pos2["geometry"] = g
pos2 = pos2.to_crs(epsg=Project.OpenStreetMaps.replace("EPSG:", ""))
m = DensityMap.get_interactive(nodes=pos2)
m
# pos2

# Global View 

Show interactive density map of the global (ground truth) for a given time.

In [ ]:

# postion collected from OMNeT 
host_position = sql1.host_position(epsg_code=Project.UTM_32N, time_slice=slice(0, 0))
host_position = host_position.to_crs(epsg=Project.OpenStreetMaps.replace("EPSG:", ""))


_i = pd.IndexSlice
# map data
cells = DensityMap.get_annotated_global_map(
    global_map = p1.global_p, 
    position = p1.position_p, 
    crs = Project.OpenStreetMaps, 
    slice_ = _i[1.0]
)
base_map = DensityMap.get_interactive(cells, host_position)
base_map


# Node View 

Show interactive density map from point of view from a single node. 

In [ ]:
# todo

# select node and time 

# extract position of 'seen' nodes from the neighborhood

# get map data from selected node